# Adsorption energy calculations using a toy calculator

A structural relaxation or structure optimization is the process of iteratively updating atom positions to find the atom positions that minimize the energy of the structure. Standard optimization methods are used in structural relaxations — below we use the Limited-Memory Broyden–Fletcher–Goldfarb–Shanno (LBFGS) algorithm. The step number, time, energy, and force max are printed at each optimization step. Each step is considered one example because it provides all the information we need to train models for the S2EF task and the entire set of steps is referred to as a trajectory. Visualizing intermediate structures or viewing the entire trajectory can be illuminating to understand what is physically happening and to look for problems in the simulation, especially when we run ML-driven relaxations. 

````{tip}
Common problems one may look out for:
* atoms excessively overlapping/colliding with each other
* atoms flying off into random directions
* the adsorbate molecule dissociating
* the adsorbate desorbing from the surface
* large changes to the surface indicating the surface is relaxed 
`````

## Setup and relaxation of a bare slab

First, let's set up a bare Cu(100) surface. We'll use ASE to make the initial structure, and LBFGS to do a quick relaxation to find the lowest energy configuration. 

We'll fix the bottom couple layers of the copper surface to approximate a very thick copper slab and prevent the surface from moving. This is a very common trick in the catalysis community. 

For this demonstration, we'll use the Effective Medium Theory (EMT) calculator from ASE. It works pretty well for simple metal structures like Cu(100) and is very fast. However, it won't do a good job with the adsorbate later on in the example!

In [ ]:
import os

import ase.io
import numpy as np
from ase import Atoms
from ase.build import fcc100
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.io import extxyz
from ase.io.trajectory import Trajectory
from ase.optimize import LBFGS

# This cell sets up and runs a structural relaxation
# of a Cu(100) surface. It uses ASE scripts to make the surface.
# The actual surfaces in OC20 were prepared slightly differently.

# Make the bare slab using an ASE helper script
slab = fcc100("Cu", size=(3, 3, 3))

# Tag all slab atoms below surface as 0, surface as 1, adsorbate as 2
tags = np.zeros(len(slab))
tags[18:27] = 1
slab.set_tags(tags)

# Fixed atoms are prevented from moving during a structure relaxation.
# We fix all slab atoms beneath the surface.
cons = FixAtoms(indices=[atom.index for atom in slab if (atom.tag == 0)])
slab.set_constraint(cons)
slab.center(vacuum=13.0, axis=2)
slab.set_pbc(True)

# Set the toy calculator (EMT) so ASE knows how to get energies/forces
# at each step
slab.set_calculator(EMT())

os.makedirs("data", exist_ok=True)

# Define structure optimizer - LBFGS. Run for 100 steps,
# or if the max force on all atoms (fmax) is below 0 ev/A.
# fmax is typically set to 0.01-0.05 eV/A,
# for this demo however we run for the full 100 steps.

dyn = LBFGS(slab, trajectory="data/Cu100.traj")
dyn.run(fmax=0.03, steps=100)

traj = ase.io.read("data/Cu100.traj", ":")

# convert traj format to extxyz format (used by OC20 dataset)
columns = ["symbols", "positions", "move_mask", "tags"]
with open("data/Cu100.extxyz", "w") as f:
    extxyz.write_xyz(f, traj, columns=columns)

### Reading a trajectory

In [ ]:
identifier = "Cu100.extxyz"

# the `index` argument corresponds to what frame of the trajectory to read in, specifiying ":" reads in the full trajectory.
traj = ase.io.read(f"data/{identifier}", index=":")

### Viewing a trajectory

Below we visualize the initial, middle, and final steps in the structural relaxation trajectory from above. Copper atoms in the surface are colored orange. EMT does a good job here and gets a relaxed structure very quickly. It only takes a few steps, and if you look closely you can see just the top layer of Cu atoms move a bit.

`````{tip}
Visualizations can be used as a quick sanity check to ensure the initial system is set up correctly and there are no major issues with the simulation!
`````

In [ ]:
import matplotlib.pyplot as plt
from ase.visualize.plot import animate
from matplotlib import rc

anim = animate(traj, radii=0.8, rotation=("-75x, 45y, 10z"))

rc("animation", html="jshtml")
anim

## Relaxation of a slab and adsorbate ("adslab")

Now that we know how to run a simple relaxation of a bare slab with ASE and a toy calculator, let's do the same thing with a methoxy (CH3O*) intermediate on the surface.

In [ ]:
import os

import ase.io
import numpy as np
from ase import Atoms
from ase.build import add_adsorbate, fcc100, molecule
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.io import extxyz
from ase.io.trajectory import Trajectory
from ase.optimize import LBFGS

# This cell sets up and runs a structural relaxation
# of a Cu(100) surface. It uses ASE scripts to make the surface.
# The actual surfaces in OC20 were prepared slightly differently.

# Make the bare slab using an ASE helper script
adslab = fcc100("Cu", size=(3, 3, 3))

# Now, let's add the adsorbate to the bare slab
adsorbate = molecule("CH3O")
add_adsorbate(adslab, adsorbate, 3, offset=(1, 1))  # adslab = adsorbate + slab

# Tag all slab atoms below surface as 0, surface as 1, adsorbate as 2
tags = np.zeros(len(adslab))
tags[18:27] = 1
tags[27:] = 2
adslab.set_tags(tags)

# Fixed atoms are prevented from moving during a structure relaxation.
# We fix all slab atoms beneath the surface.
cons = FixAtoms(indices=[atom.index for atom in adslab if (atom.tag == 0)])
adslab.set_constraint(cons)
adslab.center(vacuum=13.0, axis=2)
adslab.set_pbc(True)

# Set the toy calculator (EMT) so ASE knows how to get energies/forces
# at each step
adslab.set_calculator(EMT())

os.makedirs("data", exist_ok=True)

# Define structure optimizer - LBFGS. Run for 100 steps,
# or if the max force on all atoms (fmax) is below 0 ev/A.
# fmax is typically set to 0.01-0.05 eV/A,
# for this demo however we run for the full 100 steps.

dyn = LBFGS(adslab, trajectory="data/Cu100+CH3O.traj")
dyn.run(fmax=0.03, steps=100)

adslab_traj = ase.io.read("data/Cu100+CH3O.traj", ":")

# convert traj format to extxyz format (used by OC20 dataset)
columns = ["symbols", "positions", "move_mask", "tags"]
with open("data/Cu100+CH3O.extxyz", "w") as f:
    extxyz.write_xyz(f, adslab_traj, columns=columns)

Note that this relaxation isn't quite finished - we stopped the relaxation at 100 steps but the force is still a bit higher than we wanted. Probably it would have converged if we had waited just a little longer.

Let's visualize the relaxation with the output and see what happens!

In [ ]:
import matplotlib.pyplot as plt
from ase.visualize.plot import animate
from matplotlib import rc

anim = animate(adslab_traj, radii=0.8, rotation=("-75x, 45y, 10z"))

rc("animation", html="jshtml")
anim

### Viewing a trajectory

Below we visualize the initial, middle, and final steps in the structural relaxation trajectory from above. Copper atoms in the surface are colored orange, the propane adsorbate on the surface has grey colored carbon atoms and white colored hydrogen atoms. The adsorbate’s structure changes during the simulation and you can see how it relaxes on the surface. In this case, the relaxation looks normal; however, there can be instances where the adsorbate flies away (desorbs) from the surface or the adsorbate can break apart (dissociation), which are hard to detect without visualization. 

`````{tip}
Visualizations can be used as a quick sanity check to ensure the initial system is set up correctly and there are no major issues with the simulation!
`````

In [ ]:
import matplotlib.pyplot as plt
from ase.visualize.plot import animate
from matplotlib import rc

anim = animate(adslab_traj, radii=0.8, rotation=("-75x, 45y, 10z"))

rc("animation", html="jshtml")
anim

`````{warning}
Notice that this relaxation looks quite strange
* The oxygen is the most reactive atom here; the carbon atom is happy with four bonds (3 to H atoms, 1 to an O), but the O is clearly missing a bond. Without actually running a calculation, we could probably guess that a better configuration would be O pointing towards the copper surface. This shows that how we place the adsorbate here could be improved!
* The methoxy intermediate settles down onto the surface and falls apart (**dissociates**). The final structure is not the energy of a *OCH3 intermediate on a copper surface, but instead the energy of a collection of fragments. 

The poor relaxation is because we're using the EMT calculator, which works ok for some simple metal surfaces but is basically just a toy model for any organic atoms (C, H, O, etc). It's not a surprise that it fails for this calculation.
`````

## ASE trajectory and atoms object contents

Here we take a closer look at what information is contained within these trajectories.

In [ ]:
i_structure = adslab_traj[0]
i_structure

### Atomic numbers

In [ ]:
numbers = i_structure.get_atomic_numbers()
print(numbers)

### Atomic symbols

In [ ]:
symbols = np.array(i_structure.get_chemical_symbols())
print(symbols)

### Unit cell

The unit cell is the volume containing our system of interest. Express as a 3x3 array representing the directional vectors that make up the volume. Illustrated as the dashed box in the above visuals.

In [ ]:
cell = np.array(i_structure.cell)
print(cell)

### Periodic boundary conditions (PBC)

x,y,z boolean representing whether a unit cell repeats in the corresponding directions. The OC20 dataset sets this to [True, True, True], with a large enough vacuum layer above the surface such that a unit cell does not see itself in the z direction. This is necessary since the underlying DFT calculation also requires periodic boundary conditions. However, not all DFT codes do!

Although the original structure shown above is what gets passed into our models, the presence of PBC allows it to effectively repeat infinitely in the x and y directions. Below we visualize the same structure with a periodicity of 2 in all directions, what the model may effectively see.

In [ ]:
pbc = i_structure.pbc
print(pbc)

In [ ]:
fig, ax = plt.subplots(1, 3)
labels = ["initial", "middle", "final"]
for i in range(3):
    ax[i].axis("off")
    ax[i].set_title(labels[i])

ase.visualize.plot.plot_atoms(
    adslab_traj[0].repeat((2, 2, 1)), ax[0], radii=0.8, rotation=("-75x, 45y, 10z")
)
ase.visualize.plot.plot_atoms(
    adslab_traj[50].repeat((2, 2, 1)), ax[1], radii=0.8, rotation=("-75x, 45y, 10z")
)
ase.visualize.plot.plot_atoms(
    adslab_traj[-1].repeat((2, 2, 1)), ax[2], radii=0.8, rotation=("-75x, 45y, 10z")
)

### Tags

The OC20 dataset consists of systems with several different types of atoms. To help with identifying the index of certain atoms, we tag each atom according to where it is found in the system. There are three categories of atoms: 
- *sub-surface slab atoms*: these are atoms in the bottom layers of the catalyst, furthest away from the adsorbate
- *surface slab atoms*: these are atoms in the top layers of the catalyst, close to where the adsorbate will be placed   
- *adsorbate atoms*: atoms that make up the adsorbate molecule on top of the catalyst.

Tag definitions in OC20:

0. Sub-surface slab atoms
1. Surface slab atoms
2. Adsorbate atoms


In [ ]:
tags = i_structure.get_tags()
print(tags)

### Fixed atoms constraint

In reality, surfaces contain many, many more atoms beneath what we've illustrated as the surface. At an infinite depth, these subsurface atoms would look just like the bulk structure. We approximate a true surface by fixing the subsurface atoms into their “bulk” locations. This ensures that they cannot move at the “bottom” of the surface. If they could, this would throw off our calculations. Consistent with the above, we fix all atoms with tags=0, and denote them as "fixed". All other atoms are considered "free".

In [ ]:
cons = i_structure.constraints[0]
print(cons, "\n")

# indices of fixed atoms
indices = cons.index
print(indices, "\n")

# fixed atoms correspond to tags = 0
print(tags[indices])

### Force

Forces are another important property of the OC20 dataset. Unlike datasets like QM9 which contain only ground state properties, the OC20 dataset contains per-atom forces necessary to carry out atomistic simulations. Physically, forces are the negative gradient of energy w.r.t atomic positions: $F = -\frac{dE}{dx}$. Although not mandatory (depending on the application), maintaining this energy-force consistency is important for models that seek to make predictions on both properties.

The "apply_constraint" argument controls whether to apply system constraints to the forces. In the OC20 dataset, this controls whether to return forces for fixed atoms (apply_constraint=False) or return 0s (apply_constraint=True).

In [ ]:
# Returning forces for all atoms - regardless of whether "fixed" or "free"
i_structure.get_forces(apply_constraint=False)

In [ ]:
# Applying the fixed atoms constraint to the forces
i_structure.get_forces(apply_constraint=True)

## Adsorption (reaction) energy referenced to gas phase species

The energy of the system is one of the properties of interest in the OC20 dataset. It's important to note that absolute energies provide little value to researchers and must be referenced properly to be useful. Put another way, all physically-meaningful energies in DFT are reaction energies from one state to another. 

A common approach in catalysis to maintain consistency in energies is to reference all energies to a linear combination of specific gas phase species. For OC20, these species were N2, H2, H2O, and CO. These were chosen as the DFT method we're using does a reasonable job of calculating the energies of these species. In effect, we are calculating the energy of a specific reaction:
\begin{align*}
l\text{CO}+m\text{H2O}+n\text{H2}+p\text{N2} + * -> *OCH3
\end{align*}
where $*$ represents a catalyst surface site, and $*OCH3$ represents our intermediate bound to the catalyst surface. All chemical reactions must balance, so with a little bit of linear algebra we can figure out what numbers $l,m,n,p$ are:
\begin{align*}
\text{CO}+\frac{3}{2}\text{H2} + * -> *OCH3
\end{align*}
This is the energy that would be reported or predicted for an *OCH3 adsorbate in OC20. Another way that we could accomplish the same thing is to calculate a per-element energy for C,H,O,N from the reference energies:
\begin{align*}
E_H = E_{\text{H2}}/2 && E_N = E_{\text{N2}}/2 && E_O = E_{\text{H2O}}-E_{H2} && E_C = E_{\text{CO}}-E_{\text{H2O}}+E_{H2} 
\end{align*}
It is straightforward to convert from one reference energy scheme to another (e.g. a different set of gas-phase species), so the OC20 results are useful even if you are using a different approach!
````{danger}
It is extremely important to use consistent settings for all steps in your adsorption energy calculation. If you use different settings or codes for different species, you will get wrong numbers!
````

As a demonstration we'll do all of this using the EMT potential, but be aware that the results are going to be quite strange since EMT doesn't work for the gas phase molecules on their own!

In [ ]:
# Adslab energy
relaxed_adslab = ase.io.read("data/Cu100+CH3O.traj")
adslab_energy = relaxed_adslab.get_potential_energy()
print(f"Adsorbate+slab energy = {adslab_energy:.2f} eV")

# Corresponding raw slab used in original adslab (adsorbate+slab) system.
slab = fcc100("Cu", size=(3, 3, 3))
tags = np.zeros(len(slab))
tags[18:27] = 1
slab.set_tags(tags)
cons = FixAtoms(indices=[atom.index for atom in slab if (atom.tag == 0)])
slab.set_constraint(cons)
slab.center(vacuum=13.0, axis=2)
slab.set_calculator(EMT())
dyn = LBFGS(slab)
dyn.run(fmax=0.03, steps=100)
slab_energy = slab.get_potential_energy()
print(f"Bare slab energy = {slab_energy:.2f} eV")

# Energy for H2O
H2O_atoms = molecule("H2O")
H2O_atoms.set_pbc(True)
H2O_atoms.set_cell([20, 20, 20])
H2O_atoms.set_calculator(EMT())
dyn = LBFGS(H2O_atoms)
dyn.run(fmax=0.03, steps=100)
H2O_energy = H2O_atoms.get_potential_energy()
print(f"H2O energy = {H2O_energy:.2f} eV")

# Energy for N2
N2_atoms = molecule("N2")
N2_atoms.set_pbc(True)
N2_atoms.set_cell([20, 20, 20])
N2_atoms.set_calculator(EMT())
dyn = LBFGS(N2_atoms)
dyn.run(fmax=0.03, steps=100)
N2_energy = N2_atoms.get_potential_energy()
print(f"N2 energy = {N2_energy:.2f} eV")

# Energy for CO
CO_atoms = molecule("CO")
CO_atoms.set_pbc(True)
CO_atoms.set_cell([20, 20, 20])
CO_atoms.set_calculator(EMT())
dyn = LBFGS(CO_atoms)
dyn.run(fmax=0.03, steps=100)
CO_energy = CO_atoms.get_potential_energy()
print(f"CO energy = {CO_energy:.2f} eV")

# Energy for H2
H2_atoms = molecule("H2")
H2_atoms.set_pbc(True)
H2_atoms.set_cell([20, 20, 20])
H2_atoms.set_calculator(EMT())
dyn = LBFGS(H2_atoms)
dyn.run(fmax=0.03, steps=100)
H2_energy = H2_atoms.get_potential_energy()
print(f"H2 energy = {H2_energy:.2f} eV")

gas_reference_energies = {
    "H": H2_energy / 2,
    "N": N2_energy / 2,
    "O": H2O_energy - H2_energy,
    "C": CO_energy - (H2O_energy - H2_energy),
}

adsorbate = Atoms("CH3O").get_chemical_symbols()

adsorbate_reference_energy = 0
for ads in adsorbate:
    adsorbate_reference_energy += gas_reference_energies[ads]

print(f"Adsorbate reference energy = {adsorbate_reference_energy:.2f} eV\n")

adsorption_energy = adslab_energy - slab_energy - adsorbate_reference_energy
print(f"Adsorption energy: {adsorption_energy:.2f} eV")

`````{danger}
These gas-phase energies should not be trusted! Don't use them for anything more than this demo. Use a real calculation with consistent settings to your adslab to get the C/H/O/N energies!
`````

#### Plot energy profile of toy trajectory

Plotting the energy profile of our trajectory is a good way to ensure nothing strange has occured. We expect to see a decreasing monotonic function. If the energy is consistently increasing or there's multiple large spikes this could be a sign of some issues in the optimization. This is particularly useful for when analyzing ML-driven relaxations and whether they make general physical sense.

In [ ]:
import pandas as pd
import plotly.express as px

energies = [
    image.get_potential_energy() - slab_energy - adsorbate_reference_energy
    for image in adslab_traj
]

df = pd.DataFrame(
    {"Relaxation Step": range(len(energies)), "Adsorption Energy [eV]": energies}
)

fig = px.line(df, x="Relaxation Step", y="Adsorption Energy [eV]")
fig.show()